<a href="https://colab.research.google.com/github/Jess-cah/measure-pineapple/blob/main/detector/maskpine_160images_coco_resnet50_aug4_ALL_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This workbook goes through the training of mask r-cnn for 160 images (112/32/16 train/val/test i.e. 70/20/10 percent split) obtained from Summerpride factory.

Using COCO starting weights, RESNET50 backbone and all AUGMENTATION techniques from previous notebooks.

(i.e. FLiplr, Gaussian Noise and Blurring, as well as colour)

Use a GPU runtime for the entire workbook.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

os.getcwd()

'/content'

In [ ]:
os.chdir("/content/drive/My Drive/Colab Notebooks/Pines_Mask_RCNN/Mask_RCNN")
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/Pines_Mask_RCNN/Mask_RCNN'

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 92.5MB 44kB/s 
     |████████████████████████████████| 307kB 47.5MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 368kB 46.8MB/s 
     |████████████████████████████████| 3.2MB 45.6MB/s 
     |████████████████████████████████| 204kB 49.2MB/s 
     |████████████████████████████████| 163kB 52.4MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.1.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      S

In [ ]:
%tensorflow_version 1.x
!pip3 install Keras==2.0.8

TensorFlow 1.x selected.
     |████████████████████████████████| 276kB 9.0MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.8 which is incompatible.
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [ ]:
os.chdir("/content/drive/My Drive/Colab Notebooks/Pines_Mask_RCNN/Mask_RCNN/samples/pineappleResNet50_aug")
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/Pines_Mask_RCNN/Mask_RCNN/samples/pineappleResNet50_aug'

# Load data set

In [ ]:
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import pineapple

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

Using TensorFlow backend.


In [ ]:
config = pineapple.PineappleConfig()
PINEAPPLE_DIR = os.path.join(ROOT_DIR, "datasets/pineapple160")

In [ ]:
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE          

In [ ]:
# Training dataset
dataset_train = pineapple.PineappleDataset()
dataset_train.load_pineapple(PINEAPPLE_DIR, "train")
dataset_train.prepare()

print("Images: {}\nClasses: {}".format(len(dataset_train.image_ids), dataset_train.class_names))

Images: 112
Classes: ['BG', 'pineapple']


In [ ]:
# Validation dataset
dataset_val = pineapple.PineappleDataset()
dataset_val.load_pineapple(PINEAPPLE_DIR, "val")
dataset_val.prepare()

print("Images: {}\nClasses: {}".format(len(dataset_val.image_ids), dataset_val.class_names))

Images: 32
Classes: ['BG', 'pineapple']


# Load model and starting weights

In [ ]:
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
import tensorflow as tf

# Create model in training mode using gpu
with tf.device("/gpu:0"):
    model = modellib.MaskRCNN(mode="training", config=config,model_dir=MODEL_DIR)

# Which weights to start with?
init_with = "coco"  # imagenet, coco

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])






Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead







In [ ]:
import imgaug
import imgaug.augmenters as iaa

# set seed
imgaug.seed(1)

# define augmenter
aug = iaa.Sequential([
                      iaa.Fliplr(0.5),
                      iaa.SomeOf((0,2), [
                                         iaa.GaussianBlur(sigma=2),
                                         iaa.AdditiveGaussianNoise(loc=0, scale=0.1*255),
                                         iaa.Add((-50,50))
                      ])
])

# i.e. each image has a 50% chance of being horizontally flipped
# thereafter, between 0 and 2 of the listed operations will be performed
# Add((-50,50)) adds a random number between -50 and 50 to all pixels of a given image

In [ ]:
# Train all layers
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=30, 
            layers='all', 
            augmentation = aug)



Starting at epoch 0. LR=0.001

Checkpoint Path: /content/drive/My Drive/Colab Notebooks/Pines_Mask_RCNN/Mask_RCNN/logs/pineapple20201116T1748/mask_rcnn_pineapple_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (Bat

/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor





/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:1987: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/30
 99/100 [============================>.] - ETA: 0s - loss: 0.9070 - rpn_class_loss: 0.0473 - rpn_bbox_loss: 0.1687 - mrcnn_class_loss: 0.2242 - mrcnn_bbox_loss: 0.2719 - mrcnn_mask_loss: 0.1949

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2142: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'



100/100 [==============================] - 204s - loss: 0.9026 - rpn_class_loss: 0.0469 - rpn_bbox_loss: 0.1684 - mrcnn_class_loss: 0.2233 - mrcnn_bbox_loss: 0.2698 - mrcnn_mask_loss: 0.1941 - val_loss: 0.3015 - val_rpn_class_loss: 0.0049 - val_rpn_bbox_loss: 0.0478 - val_mrcnn_class_loss: 0.0902 - val_mrcnn_bbox_loss: 0.0603 - val_mrcnn_mask_loss: 0.0983
Epoch 2/30
100/100 [==============================] - 97s - loss: 0.2990 - rpn_class_loss: 0.0065 - rpn_bbox_loss: 0.0456 - mrcnn_class_loss: 0.0961 - mrcnn_bbox_loss: 0.0546 - mrcnn_mask_loss: 0.0962 - val_loss: 0.2521 - val_rpn_class_loss: 0.0037 - val_rpn_bbox_loss: 0.0455 - val_mrcnn_class_loss: 0.0796 - val_mrcnn_bbox_loss: 0.0409 - val_mrcnn_mask_loss: 0.0824
Epoch 3/30
100/100 [==============================] - 97s - loss: 0.2632 - rpn_class_loss: 0.0053 - rpn_bbox_loss: 0.0366 - mrcnn_class_loss: 0.0935 - mrcnn_bbox_loss: 0.0404 - mrcnn_mask_loss: 0.0874 - val_loss: 0.2294 - val_rpn_class_loss: 0.0034 - val_rpn_bbox_loss: 0.0